In [2]:
import torch

torch.cuda.is_available()

False

In [3]:
from ultralytics import YOLO

In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov8n.pt")

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO inference
    results = model.predict(frame)

    # Visualize results on the frame
    annotated_frame = results[0].plot()

    # Show the frame
    cv2.imshow("YOLO Inference", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 3 persons, 1 chair, 1 couch, 66.4ms
Speed: 1.7ms preprocess, 66.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 chair, 44.7ms
Speed: 2.5ms preprocess, 44.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 chair, 52.3ms
Speed: 1.0ms preprocess, 52.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 chair, 44.7ms
Speed: 0.0ms preprocess, 44.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 chair, 45.9ms
Speed: 0.0ms preprocess, 45.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 chair, 46.9ms
Speed: 1.1ms preprocess, 46.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 chair, 45.4ms
Speed: 1.0ms preprocess, 45.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 chair, 50.6ms


In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch
import numpy as np

def visualize_model_architecture():
    fig, ax = plt.subplots(1, 1, figsize=(16, 12))
    ax.set_xlim(0, 16)
    ax.set_ylim(0, 14)
    ax.axis('off')
    
    # Define colors
    colors = {
        'input': '#E8F4FD',
        'backbone': '#B3D9FF', 
        'encoder': '#66B2FF',
        'attention': '#1A8CFF',
        'fusion': '#0066CC',
        'classifier': '#004C99',
        'output': '#003366'
    }
    
    # Helper function to add rounded rectangle
    def add_box(ax, x, y, width, height, text, color, text_size=10):
        box = FancyBboxPatch((x, y), width, height, 
                           boxstyle="round,pad=0.1", 
                           facecolor=color, 
                           edgecolor='black', 
                           linewidth=1.5)
        ax.add_patch(box)
        ax.text(x + width/2, y + height/2, text, 
               ha='center', va='center', fontsize=text_size, weight='bold')
    
    # Helper function to add arrow
    def add_arrow(ax, x1, y1, x2, y2, color='black'):
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', lw=2, color=color))
    
    # Title
    ax.text(8, 13.5, 'DualBackboneTimeLapseClassifier Architecture', 
           ha='center', va='center', fontsize=16, weight='bold')
    
    # Input layers
    add_box(ax, 1, 11.5, 2.5, 1, 'Context Frames\n(B×5×3×H×W)', colors['input'], 9)
    add_box(ax, 12.5, 11.5, 2.5, 1, 'Reference Frames\n(B×3×3×H×W)', colors['input'], 9)
    
    # Backbone encoders
    add_box(ax, 1, 9.5, 2.5, 1.5, 'EfficientNet-B0\nContext Backbone\n(3 channels)', colors['backbone'], 9)
    add_box(ax, 12.5, 9.5, 2.5, 1.5, 'EfficientNet-B0\nReference Backbone\n(6 channels)', colors['backbone'], 9)
    
    # Feature compression
    add_box(ax, 1, 7.5, 2.5, 1, 'Linear Compression\nto 512 dims', colors['encoder'], 9)
    add_box(ax, 12.5, 7.5, 2.5, 1, 'Linear Compression\nto 512 dims', colors['encoder'], 9)
    
    # Feature reshaping
    add_box(ax, 1, 5.5, 2.5, 1, 'Reshape to\n(B×5×512)', colors['encoder'], 9)
    add_box(ax, 12.5, 5.5, 2.5, 1, 'Reshape to\n(B×1×512)', colors['encoder'], 9)
    
    # Concatenation
    add_box(ax, 6, 4, 4, 1, 'Concatenate Features\n(B×6×512)', colors['fusion'], 10)
    
    # Positional encoding
    add_box(ax, 6, 2.5, 4, 1, 'Positional Encoding\n+ Transformer Encoder\n(4 layers, 8 heads)', colors['attention'], 10)
    
    # Feature separation and pooling
    add_box(ax, 2, 1, 3, 1, 'Context Features\nMean Pooling\n(B×512)', colors['fusion'], 9)
    add_box(ax, 11, 1, 3, 1, 'Reference Features\nLast Token\n(B×512)', colors['fusion'], 9)
    
    # Final fusion and classification
    add_box(ax, 6, -0.5, 4, 1, 'Feature Concatenation\n(B×1024)', colors['fusion'], 10)
    add_box(ax, 6.5, -2, 3, 1, 'MLP Classifier\n1024→512→256→2', colors['classifier'], 10)
    add_box(ax, 7, -3.5, 2, 0.8, 'Output\n(B×2)', colors['output'], 10)
    
    # Add arrows for data flow
    # Context path
    add_arrow(ax, 2.25, 11.5, 2.25, 11)  # Input to backbone
    add_arrow(ax, 2.25, 9.5, 2.25, 8.5)  # Backbone to compression
    add_arrow(ax, 2.25, 7.5, 2.25, 6.5)  # Compression to reshape
    add_arrow(ax, 3.5, 6, 6, 4.8)        # Reshape to concat
    
    # Reference path  
    add_arrow(ax, 13.75, 11.5, 13.75, 11)  # Input to backbone
    add_arrow(ax, 13.75, 9.5, 13.75, 8.5)  # Backbone to compression
    add_arrow(ax, 13.75, 7.5, 13.75, 6.5)  # Compression to reshape
    add_arrow(ax, 12.5, 6, 10, 4.8)        # Reshape to concat
    
    # Main flow
    add_arrow(ax, 8, 4, 8, 3.5)      # Concat to transformer
    add_arrow(ax, 6.5, 2.5, 3.5, 2)  # Transformer to context pooling
    add_arrow(ax, 9.5, 2.5, 12.5, 2) # Transformer to reference pooling
    add_arrow(ax, 3.5, 1, 6, 0.3)    # Context pooling to final fusion
    add_arrow(ax, 12.5, 1, 10, 0.3)  # Reference pooling to final fusion
    add_arrow(ax, 8, -0.5, 8, -1)    # Fusion to classifier
    add_arrow(ax, 8, -2, 8, -2.7)    # Classifier to output
    
    # Add legend
    legend_elements = [
        patches.Patch(color=colors['input'], label='Input'),
        patches.Patch(color=colors['backbone'], label='Backbone'),
        patches.Patch(color=colors['encoder'], label='Feature Processing'),
        patches.Patch(color=colors['attention'], label='Transformer'),
        patches.Patch(color=colors['fusion'], label='Feature Fusion'),
        patches.Patch(color=colors['classifier'], label='Classification'),
        patches.Patch(color=colors['output'], label='Output')
    ]
    ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))
    
    plt.tight_layout()
    plt.show()

# Call the visualization function
visualize_model_architecture()

: 